In [14]:
%pip install torch
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.data import RandomSampler
from torchvision.datasets import ImageFolder

# Define paths to the criminal face dataset and the general face dataset
criminal_train_path = 'C:/Users/prasa/Desktop/4-2/face/face/aligned_faces/train'
criminal_val_path = 'C:/Users/prasa/Desktop/4-2/face/face/aligned_faces/val'
criminal_test_path = 'C:/Users/prasa/Desktop/4-2/face/face/aligned_faces/test'

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Add random rotation
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Add color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load criminal face datasets with data augmentation
criminal_train_dataset = ImageFolder(root=criminal_train_path, transform=transform)

# Calculate class weights for handling class imbalance
class_weights = torch.zeros(len(criminal_train_dataset.classes))
for _, label in criminal_train_dataset:
    class_weights[label] += 1
class_weights = 1 / class_weights
sample_weights = [class_weights[label] for _, label in criminal_train_dataset]

# Use weighted random sampler for handling class imbalance
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# Define data loaders for training, validation, and testing sets
batch_size = 32
criminal_train_loader = DataLoader(criminal_train_dataset, batch_size=batch_size, sampler=sampler)
criminal_val_dataset = ImageFolder(root=criminal_val_path, transform=transform)
criminal_val_loader = DataLoader(criminal_val_dataset, batch_size=batch_size)
criminal_test_dataset = ImageFolder(root=criminal_test_path, transform=transform)
criminal_test_loader = DataLoader(criminal_test_dataset, batch_size=batch_size)



class FaceSwinIR(nn.Module):
    def __init__(self, num_classes):
        super(FaceSwinIR, self).__init__()
        # Define the layers of the FaceSwinIR model
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 16 * 16, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 512 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
# Initialize the FaceSwinIR model with the number of classes in the custom dataset
num_classes_custom = 10  # Update with the actual number of classes in the custom dataset
model_custom = FaceSwinIR(num_classes_custom)

# Define loss function and optimizer for custom dataset
criterion_custom = nn.CrossEntropyLoss()
optimizer_custom = optim.Adam(model_custom.parameters(), lr=0.001)

# Training loop for custom dataset
num_epochs_custom = 30
best_val_loss_custom = float('inf')
for epoch in range(num_epochs_custom):
    # Training
    model_custom.train()
    train_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    for images, labels in criminal_train_loader:
        optimizer_custom.zero_grad()
        outputs = model_custom(images)
        loss = criterion_custom(outputs, labels)
        loss.backward()
        optimizer_custom.step()
        train_loss_custom += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_custom += labels.size(0)
        correct_custom += (predicted == labels).sum().item()
    train_loss_custom /= len(criminal_train_loader.dataset)
    train_accuracy_custom = correct_custom / total_custom

    # Validation
    model_custom.eval()
    val_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    with torch.no_grad():
        for images, labels in criminal_val_loader:
            outputs = model_custom(images)
            loss = criterion_custom(outputs, labels)
            val_loss_custom += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_custom += labels.size(0)
            correct_custom += (predicted == labels).sum().item()
        val_loss_custom /= len(criminal_val_loader.dataset)
        val_accuracy_custom = correct_custom / total_custom

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs_custom}], '
          f'Train Loss: {train_loss_custom:.4f}, '
          f'Train Accuracy: {train_accuracy_custom:.2%}, '
          f'Val Loss: {val_loss_custom:.4f}, '
          f'Val Accuracy: {val_accuracy_custom:.2%}')

# Save the trained model
torch.save(model_custom.state_dict(), 'criminal_face_detection_model.pth')

ZeroDivisionError: division by zero

In [7]:
# Define testing loop for custom dataset
model_custom.eval()
test_loss_custom = 0.0
correct_custom = 0
total_custom = 0
with torch.no_grad():
    for images, labels in criminal_test_loader:
        outputs = model_custom(images)
        loss = criterion_custom(outputs, labels)
        test_loss_custom += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_custom += labels.size(0)
        correct_custom += (predicted == labels).sum().item()
test_loss_custom /= len(criminal_test_loader.dataset)
test_accuracy_custom = correct_custom / total_custom

print(f'Test Loss on Custom Dataset: {test_loss_custom:.4f}')
print(f'Test Accuracy on Custom Dataset: {test_accuracy_custom:.2%}')

Test Loss on Custom Dataset: 2.3002
Test Accuracy on Custom Dataset: 22.86%


In [1]:
%pip install keras_vggface
%pip install keras_applications
%pip install keras==2.2.4
%pip install tensorflow==1.15.5

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.5 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==1.15.5


In [3]:
import cv2
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

class VGGFaceRecognizer(nn.Module):
    def __init__(self, num_classes):
        super(VGGFaceRecognizer, self).__init__()
        # Load pre-trained VGGFace model
        self.vggface = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
        # Freeze the VGGFace layers
        for param in self.vggface.parameters():
            param.requires_grad = False
        # Add fully connected layers for classification
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = preprocess_input(x)  # Preprocess input for VGGFace
        x = self.vggface(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

# Define paths to the criminal face dataset
criminal_train_path = 'C:/crowdcrime/yolov8-streamlit-detection-tracking/face/aligned_faces/train'
criminal_val_path = 'C:/crowdcrime/yolov8-streamlit-detection-tracking/face/aligned_faces/val'

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for VGGFace
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for VGGFace
])

# Load criminal face datasets with data augmentation
criminal_train_dataset = ImageFolder(root=criminal_train_path, transform=transform)
criminal_val_dataset = ImageFolder(root=criminal_val_path, transform=transform)

# Define data loaders for training and validation sets
batch_size = 32
criminal_train_loader = DataLoader(criminal_train_dataset, batch_size=batch_size, shuffle=True)
criminal_val_loader = DataLoader(criminal_val_dataset, batch_size=batch_size)

# Initialize the VGGFaceRecognizer model with the number of classes in the custom dataset
num_classes_custom = len(criminal_train_dataset.classes)
model_custom = VGGFaceRecognizer(num_classes_custom)

# Define loss function and optimizer for custom dataset
criterion_custom = nn.CrossEntropyLoss()
optimizer_custom = optim.Adam(model_custom.parameters(), lr=0.001)

# Training loop for custom dataset
num_epochs_custom = 30
for epoch in range(num_epochs_custom):
    # Training
    model_custom.train()
    train_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    for images, labels in criminal_train_loader:
        optimizer_custom.zero_grad()
        outputs = model_custom(images)
        loss = criterion_custom(outputs, labels)
        loss.backward()
        optimizer_custom.step()
        train_loss_custom += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_custom += labels.size(0)
        correct_custom += (predicted == labels).sum().item()
    train_loss_custom /= len(criminal_train_loader.dataset)
    train_accuracy_custom = correct_custom / total_custom

    # Validation
    model_custom.eval()
    val_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    with torch.no_grad():
        for images, labels in criminal_val_loader:
            outputs = model_custom(images)
            loss = criterion_custom(outputs, labels)
            val_loss_custom += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_custom += labels.size(0)
            correct_custom += (predicted == labels).sum().item()
        val_loss_custom /= len(criminal_val_loader.dataset)
        val_accuracy_custom = correct_custom / total_custom

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs_custom}], '
          f'Train Loss: {train_loss_custom:.4f}, '
          f'Train Accuracy: {train_accuracy_custom:.2%}, '
          f'Val Loss: {val_loss_custom:.4f}, '
          f'Val Accuracy: {val_accuracy_custom:.2%}')

# Save the trained model
torch.save(model_custom.state_dict(), 'criminal_face_recognition_model.pth')

Using TensorFlow backend.
C:\Users\vanam\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3110: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if training is 1 or training is True:
C:\Users\vanam\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3116: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif training is 0 or training is False:
C:\Users\vanam\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3110: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if training is 1 or training is True:
C:\Users\vanam\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3116: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif training is 0 or training is False:


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from keras_vggface.vggface import VGGFace

# Define paths to the criminal face dataset and the general face dataset
criminal_train_path = 'C:/crowdcrime/yolov8-streamlit-detection-tracking/face/aligned_faces/train'
criminal_val_path = 'C:/crowdcrime/yolov8-streamlit-detection-tracking/face/aligned_faces/val'
criminal_test_path = 'C:/crowdcrime/yolov8-streamlit-detection-tracking/face/aligned_faces/test'

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing for VGGFace input
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Add random rotation
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Add color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load criminal face datasets with data augmentation
criminal_train_dataset = ImageFolder(root=criminal_train_path, transform=transform)

# Calculate class weights for handling class imbalance
class_weights = torch.zeros(len(criminal_train_dataset.classes))
for _, label in criminal_train_dataset:
    class_weights[label] += 1
class_weights = 1 / class_weights
sample_weights = [class_weights[label] for _, label in criminal_train_dataset]

# Use weighted random sampler for handling class imbalance
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# Define data loaders for training, validation, and testing sets
batch_size = 32
criminal_train_loader = DataLoader(criminal_train_dataset, batch_size=batch_size, sampler=sampler)
criminal_val_dataset = ImageFolder(root=criminal_val_path, transform=transform)
criminal_val_loader = DataLoader(criminal_val_dataset, batch_size=batch_size)
criminal_test_dataset = ImageFolder(root=criminal_test_path, transform=transform)
criminal_test_loader = DataLoader(criminal_test_dataset, batch_size=batch_size)

class VGGFaceRecognizer(nn.Module):
    def __init__(self, num_classes):
        super(VGGFaceRecognizer, self).__init__()
        # Load pre-trained VGGFace model
        self.vggface = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
        # Freeze the VGGFace layers
        for param in self.vggface.parameters():
            param.requires_grad = False
        # Define additional layers for classification
        self.fc = nn.Linear(2048, num_classes)  # ResNet50 outputs 2048 features

    def forward(self, x):
        # Pass input through VGGFace model
        x = self.vggface(x)
        # Perform classification
        x = self.fc(x)
        return x

# Initialize the VGGFaceRecognizer model with the number of classes in the custom dataset
num_classes_custom = len(criminal_train_dataset.classes)
model_custom = VGGFaceRecognizer(num_classes_custom)

# Define loss function and optimizer for custom dataset
criterion_custom = nn.CrossEntropyLoss()
optimizer_custom = optim.Adam(model_custom.parameters(), lr=0.001)

# Training loop for custom dataset
num_epochs_custom = 30
best_val_loss_custom = float('inf')
for epoch in range(num_epochs_custom):
    # Training
    model_custom.train()
    train_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    for images, labels in criminal_train_loader:
        optimizer_custom.zero_grad()
        outputs = model_custom(images)
        loss = criterion_custom(outputs, labels)
        loss.backward()
        optimizer_custom.step()
        train_loss_custom += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_custom += labels.size(0)
        correct_custom += (predicted == labels).sum().item()
    train_loss_custom /= len(criminal_train_loader.dataset)
    train_accuracy_custom = correct_custom / total_custom

    # Validation
    model_custom.eval()
    val_loss_custom = 0.0
    correct_custom = 0
    total_custom = 0
    with torch.no_grad():
        for images, labels in criminal_val_loader:
            outputs = model_custom(images)
            loss = criterion_custom(outputs, labels)
            val_loss_custom += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_custom += labels.size(0)
            correct_custom += (predicted == labels).sum().item()
        val_loss_custom /= len(criminal_val_loader.dataset)
        val_accuracy_custom = correct_custom / total_custom

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs_custom}], '
          f'Train Loss: {train_loss_custom:.4f}, '
          f'Train Accuracy: {train_accuracy_custom:.2%}, '
          f'Val Loss: {val_loss_custom:.4f}, '
          f'Val Accuracy: {val_accuracy_custom:.2%}')

# Save the trained model
torch.save(model_custom.state_dict(), 'criminal_face_detection_model.pth')

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'